In [1]:
import numpy as np
from PIL import Image
from skimage.transform import rescale
import itertools
import math
import time
from datetime import datetime as dt
import collections

def get_input(day, split=None, f=str.strip):
    fin = f'2021-{day}.txt'
    if split is None:
        input = open(fin, 'r').readlines()
    else:
        input = open(fin, 'r').read().split(split)
    
    if f is not None:
        input = [f(x) for x in input]
    
    return input

def timestamp(ts):
    date = dt.fromtimestamp(ts)
    return f'[{date.hour:02}:{date.minute:02}:{date.second:02}]'

def output(*args, ts=0):
    out = ' '.join([str(x) for x in args])
    if ts:
        ts = timestamp(ts)
        sz = 76 - len(out) - len(ts)
        if sz > 0:
            out += ' ' * sz
        out += ts
    print(out)

# Day 1

In [2]:
input = get_input(1, f=int)

def depth_inc(lst, win=1):
    cnt = 0
    for i in range(len(lst) - win):
        if lst[i+win] > lst[i]:
            cnt += 1
    return cnt

output(depth_inc(input), ts=1638334897)
output(depth_inc(input, 3), ts=1638335082)

1688                                                              [00:01:37]
1728                                                              [00:04:42]


# Day 2

In [3]:
input = get_input(2)

def find_pos(lst, use_aim=True):
    depth = pos = aim = 0
    for item in lst:
        cmd, X = item.split()
        X = int(X)
        if cmd == 'forward':
            pos += X
            if use_aim:
                depth += X * aim
        elif cmd == 'down':
            if use_aim:
                aim += X
            else:
                depth += X
        elif cmd == 'up':
            if use_aim:
                aim -= X
            else:
                depth -= X
    return depth * pos

output(find_pos(input, False), ts=1638421338)
output(find_pos(input, True), ts=1638421474)

1670340                                                           [00:02:18]
1954293920                                                        [00:04:34]


# Day 3

In [4]:
input = get_input(3)

cnt = [0] * (len(input[0]))
for x in input:
    for i,v in enumerate(x):
        cnt[i] += int(v)

sz = len(input) / 2
gamma = epsilon = ''
for x in cnt:
    if x > sz:
        gamma += '1'
        epsilon += '0'
    else:
        gamma += '0'
        epsilon += '1'
gamma = int(gamma, 2)
epsilon = int(epsilon, 2)

output(gamma*epsilon, ts=1638508079) #1 3969000


def bin_search(lst, pos, most=True):
    cnt = 0
    ones = []
    zeros = []
    for x in lst:
        if int(x[pos]):
            cnt += 1
            ones.append(x)
        else:
            zeros.append(x)
    
    if cnt >= len(lst) / 2:
        if most:
            out = ones
        else:
            out = zeros
    elif most:
        out = zeros
    else:
        out = ones
    
    if len(out) == 1:
        return int(out[0], 2)
    
    return bin_search(out, pos+1, most)


o2  = bin_search(input, 0, True)
co2 = bin_search(input, 0, False)
output(o2 * co2, ts=1638508559) #2 4267809

3969000                                                           [00:07:59]
4267809                                                           [00:15:59]


# Day 4

In [5]:
input = get_input(4)

calls = [int(x) for x in input[0].split(',')]

boards = []
for i in range(2, len(input)-4, 6):
    b = []
    for j in range(5):
        b.append([int(x) for x in input[i+j].split()])
    boards.append(np.asarray(b))

def bingo(board):
    for row in board:
        if np.sum(row) == -5:
            return True
    for col in board.T:
        if np.sum(col) == -5:
            return True
    return False

def score(board, call):
    board[np.where(board==-1)] = 0
    return np.sum(board) * call

cnt = 0
last = len(boards)
for x in calls:
    hit = False
    for b in boards:
        if x in b:
            b[np.where(b==x)] = -1
            if bingo(b):
                cnt += 1
                if cnt == 1:
                    output(score(b,x), ts=1638594977) #1
                elif cnt == last:
                    output(score(b,x), ts=1638595294) #2
                b[:][:] = -99

8580                                                              [00:16:17]
9576                                                              [00:21:34]


# Day 5

In [6]:
def f(s):
    a, _, b = s.split()
    x1, y1 = [int(i) for i in a.split(',')]
    x2, y2 = [int(i) for i in b.split(',')]  
    return x1, y1, x2, y2

input = get_input(5, f=f)

vents = np.zeros((1000,1000), np.int32)
cnt = np.ones((1000,1000), np.int32)

for x1, y1, x2, y2 in input:
    if x1 == x2:
        if y1 > y2:
            r = range(y2, y1+1)
        else:
            r = range(y1, y2+1)
        for y in r:
            vents[x1][y] += 1
    elif y1 == y2:
        if x1 > x2:
            r = range(x2, x1+1)
        else:
            r = range(x1, x2+1)
        for x in r:
            vents[x][y1] += 1

output(np.sum(cnt[np.where(vents > 1)]), ts=1638681085) #1

for x1, y1, x2, y2 in input:
    if x1 != x2 and y1 != y2:
        if x1 < x2:
            r = range(x1, x2+1)
        else:
            r = range(x1, x2-1, -1)
        
        if y1 < y2:
            f = lambda a,b: a+b
        else:
            f = lambda a,b: a-b
        
        for y,x in enumerate(r):
            vents[x][f(y1,y)] += 1

output(np.sum(cnt[np.where(vents > 1)]), ts=1638682511) #2

7436                                                              [00:11:25]
21104                                                             [00:35:11]


# Day 6

In [7]:
input = get_input(6, split=',', f=int)

fish = collections.Counter(input) # count fish at each time

def epoch(a):
    b = {}              # next epoch
    a[7] = a[7] + a[0]  # reset 0 -> 6 (subtract below)
    b[8] = a[0]         # add new fish (not subtracted)
    for i in range(8):  # subtract 1 from each
        b[i] = a[i+1]
    return b

for day in range(80):
    fish = epoch(fish)

assert sum(fish.values()) == 350149
output(sum(fish.values()), ts=1638767460) #1

for day in range(256-80):
    fish = epoch(fish)
    
assert sum(fish.values()) == 1590327954513
output(sum(fish.values()), ts=1638768282) #2

350149                                                            [00:11:00]
1590327954513                                                     [00:24:42]


# Day 7

In [8]:
input = get_input(7, split=',', f=int)

low_lin = low_geom = 10**20
for i in range(max(input)):
    cost_lin = cost_geom = 0
    for x in input:
        cost_lin += abs(x-i)
        d = abs(x-i)
        cost_geom += (d*d+d)//2
    if cost_lin < low_lin:
        low_lin = cost_lin
    if cost_geom < low_geom:
        low_geom = cost_geom
        
output(low_lin, ts=1638853498)
output(low_geom, ts=1638853947)

356992                                                            [00:04:58]
101268110                                                         [00:12:27]


# Day 8

In [11]:
input = get_input(8)

outs = [x.split('|')[1] for x in input]

cnt = 0
for x in outs:
    for y in x.split():
        if len(y) in [2,3,4,7]:
            cnt += 1

assert cnt == 365
output(cnt, ts=1638940069) #1

cnt = 0
for x in input:
    ins, outs = x.split('|')
    ins = ins.split()
    a = b = c = d = e = f = g = ''
    
    # find easy nums by length
    for y in ins:
        if len(y) == 2:
            one = y
        elif len(y) == 3:
            seven = y
        elif len(y) == 4:
            four = y
        elif len(y) == 7:
            eight = y
    # find three and six using one
    for y in ins:
        if len(y) == 5 and one[0] in y and one[1] in y:
            three = y
        elif len(y) == 6 and not (one[0] in y and one[1] in y):
            six = y
    # a - in seven but not one
    for i in seven:
        if i not in one:
            a = i
    # b - in four but not three
    for i in four:
        if i not in three:
            b = i
    # c - in eight but not six
    for i in eight:
        if i not in six:
            c = i
    # d - in three and four but not one
    # g - in three but not four (and not a)
    for i in three:
        if i in four and i not in one:
            d = i
        elif i not in four and i != a:
            g = i
    # e - in six but not three (and not b)
    # f - in six and one
    for i in six:
        if i not in three and i != b:
            e = i
        elif i in one:
            f = i
    
    if '' in [a,b,c,d,e,f,g]:
        print('shiza', ins)
        continue
    
    out = ''
    for y in outs.split():
        if len(y) == 6 and d not in y:
            out += '0'
        elif len(y) == 2:
            out += '1'
        elif len(y) == 5 and e in y:
            out += '2'
        elif len(y) == 5 and c in y and f in y:
            out += '3'
        elif len(y) == 4:
            out += '4'
        elif len(y) == 5 and b in y:
            out += '5'
        elif len(y) == 6 and c not in y:
            out += '6'
        elif len(y) == 3:
            out += '7'
        elif len(y) == 7:
            out += '8'
        elif len(y) == 6 and e not in y:
            out += '9'
        else:
            print('shiza', y)
    #print(out)
    
    cnt += int(out)

assert cnt == 975706
output(cnt, ts=1638942090) #2

365                                                               [00:07:49]
975706                                                            [00:41:30]


# Day 9

In [12]:
input = get_input(9)

output()